In [40]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from fredapi import Fred
import statsmodels.api as sm
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
fred = Fred(api_key='YOUR_API_KEY_HERE')
from sklearn.linear_model import ElasticNetCV, ElasticNet

In [41]:
os.chdir("C:/Users/gabeyie/OneDrive - University of Tennessee/Documents/IJF_Paper")
os.makedirs('Forecasts', exist_ok=True)
print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Users\gabeyie\OneDrive - University of Tennessee\Documents\IJF_Paper


In [42]:
with open("complete_words.json", "r") as file:
    complete_words = json.load(file)

In [43]:
oil_price = fred.get_series('DCOILWTICO', observation_start='1986-01-01', observation_end='2020-12-01')
monthly_oil = oil_price.resample('ME').mean().ffill()
monthly_oil_log = np.log(monthly_oil)
monthly_oil_log_diff = monthly_oil_log.diff()
monthly_oil_log_diff = monthly_oil_log_diff.dropna()
Target = monthly_oil_log_diff.values.ravel()
Lag_Target = monthly_oil_log_diff.shift(1)
Lag_Target = Lag_Target.bfill()
Lag_Target = Lag_Target.values.ravel()

In [44]:
Features = pd.read_excel('Data/revise.xlsx')
Quant_Features = pd.read_excel('Data/EMV_Data.xlsx')

# Create the "Date" column with yyyy/mm/dd format, setting the day to "01"
Quant_Features['Date'] = pd.to_datetime(Quant_Features['Year'].astype(str) + '/' + Quant_Features['Month'].astype(str) + '/01')

In [45]:
Quant_Features['Date'] = pd.to_datetime(Quant_Features['Date'])
Quant_Features.set_index('Date', inplace=True)

# Selecting the desired date range
selected_df = Quant_Features.loc['1986-02-01':'2020-12-01']

In [46]:
# Get the Sahm Rule Recession Indicator
sahn_index = fred.get_series('SAHMREALTIME', observation_start='1986-02-01', observation_end='2020-12-01')

# Initialize recession_expansion and update as matrices of True
recession_expansion = pd.DataFrame(True, index=sahn_index.index, columns=['indicator'])
update = recession_expansion.copy()

# Update recession_expansion: set to False if Sahm Rule Recession Indicator > 0.5
recession_expansion.loc[sahn_index > 0.5, 'indicator'] = False

# Update the update matrix: set to True only for the first time point and whenever the value of recession_expansion changes
update['indicator'] = recession_expansion['indicator'].ne(recession_expansion['indicator'].shift())
update.loc[update.index[0], 'indicator'] = True
updates = update.values.ravel()

In [47]:
train_ratio = 0.625
n_samples = len(monthly_oil_log_diff)
n_train = int(n_samples * train_ratio)
updates = update.values.ravel()

In [48]:
Numeric_data = np.array(selected_df)

Numeric_data_train = Numeric_data[:n_train]
Numeric_data_test = Numeric_data[n_train:]

In [49]:
# Create a list to store dictionaries
dict_list = []

# Iterate over rows in the original dataframe
for index, row in Features.iterrows():
    # Get the text for the current row
    text = row['cleaned_Text']
    
    # Split the text into words
    words = text.split()
    
    # Count the frequency of each word
    word_counts = Counter(words)
    
    # Initialize a dictionary to hold the counts for the words in your list
    count_dict = {}
    
    # For each word in your list, get its count and add to the dictionary
    for word in complete_words:
        count_dict[word] = word_counts.get(word, 0)
    
    # Add the dictionary to the list
    dict_list.append(count_dict)

# Convert the list of dictionaries into a dataframe
df_counts = pd.DataFrame(dict_list)

In [50]:
Fixed = df_counts
split_ratio = 0.625
split_index = int(len(Fixed)*split_ratio)
Fixed_train = Fixed.iloc[:split_index]
Fixed_test = Fixed.iloc[split_index:]
Fixed_train = Fixed_train.iloc[1:, :]

# Convert all training sets to arrays
Fixed_train = np.array(Fixed_train)

# Convert all testing sets to arrays
Fixed_test = np.array(Fixed_test)


Benchmark_datasets1 = {
    r'TF-IDF Colls($D_{1,t}$)': (Fixed_train, Fixed_test),
    r'Noun-Noun/Adj-Noun Colls($D_{2,t}$)': (Fixed_train, Fixed_test),
    r'Verb-Noun/Noun-Verb Colls($D_{3,t}$)': (Fixed_train, Fixed_test),
    r'Noun-Adj/Verb-Adj Colls($D_{4,t}$)': (Fixed_train, Fixed_test)      
}

Benchmark_datasets2 = {
    r'TF-IDF Colls($D_{1,t}$)': (Numeric_data_train, Numeric_data_test),
    r'Noun-Noun/Adj-Noun Colls($D_{2,t}$)': (Numeric_data_train, Numeric_data_test),
    r'Verb-Noun/Noun-Verb Colls($D_{3,t}$)': (Numeric_data_train, Numeric_data_test),
    r'Noun-Adj/Verb-Adj Colls($D_{4,t}$)': (Numeric_data_train, Numeric_data_test)    
}

In [51]:
def num_factors(data, kmax):
    T, N = data.shape
    K = min(kmax, N)

    xx = (data.T @ data) / (T*N) if N < T else (data @ data.T) / (T*N)

    eig_values = np.linalg.eigvals(xx)
    d = sorted(eig_values, reverse=True)

    ER = [d[k] / d[k+1] for k in range(K-1)]
    ER = [0 if np.isnan(e) or np.isinf(e) else e for e in ER]
    
    n_fac = max(ER)
    
    num_factors = ER.index(n_fac) + 1 # Remember python indexing starts from 0 so +1

    return num_factors

In [52]:
# Initialize your objects
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits)
scaler = StandardScaler()
horizons = [1, 3, 6, 9]
predictions_dict_bm = {}
y_true_dict_bm = {}

for model_name, (train, test) in Benchmark_datasets1.items():
    predictions_dict_bm[model_name] = {h: [] for h in horizons}
    y_true_dict_bm[model_name] = {h: [] for h in horizons}

    data = np.concatenate([train, test])

    for h in horizons:
        y_true_per_bm_horizon = []
        y_pred_per_bm_horizon = []

        for i in range(len(train) + h - 1, len(data)):

            train_temp = data[:i - h + 1]
            test_temp = data[i - h + 1:i + 1]

            # Get the corresponding targets
            y_train_temp = Target[:i - h + 1]
            y_test_temp = Target[i - h + 1:i + 1]

            # Get the corresponding lag targets
            y_lag_train_temp = Lag_Target[:i - h + 1]
            y_lag_test_temp = Lag_Target[i - h + 1:i + 1]

            scaler.fit(train_temp)
            train_temp_standardized = scaler.transform(train_temp)
            test_temp_standardized = scaler.transform(test_temp)
            
            n_components = num_factors(train_temp_standardized, kmax=8)
            pca = PCA(n_components=n_components)
            pca.fit(train_temp_standardized)

            selected_train_temp_pca = pca.transform(train_temp_standardized)
            selected_test_temp_pca = pca.transform(test_temp_standardized)

            # Add the lagged target as an additional column to the PCA-transformed data
            selected_train_temp_pca = np.column_stack((selected_train_temp_pca, y_lag_train_temp))
            selected_test_temp_pca = np.column_stack((selected_test_temp_pca, y_lag_test_temp))

            lr = LinearRegression()
            mod = sm.OLS(np.ravel(y_train_temp), sm.add_constant(selected_train_temp_pca))
            fii = mod.fit()
            p_values = fii.summary2().tables[1]['P>|t|']

            significant_features = [idx for idx, p in enumerate(p_values[1:]) if p < 0.05]

            if significant_features:
                selected_train_temp_pca = selected_train_temp_pca[:, significant_features]
                selected_test_temp_pca = selected_test_temp_pca[:, significant_features]
            else:
                print("No features with p-value < 0.05 found. Retaining all PCA-transformed features.")

            lr.fit(selected_train_temp_pca, np.ravel(y_train_temp))

            # Predict using the freshly fitted model
            y_pred_pca_temp = lr.predict(selected_test_temp_pca)
            y_pred_per_bm_horizon.append(y_pred_pca_temp[h-1])

            y_true_per_bm_horizon.append(y_test_temp[h-1])

        predictions_dict_bm[model_name][h] = y_pred_per_bm_horizon
        y_true_dict_bm[model_name][h] = y_true_per_bm_horizon

with open('Forecasts/predictions_dict_bm1.pkl', 'wb') as f:
    pickle.dump(predictions_dict_bm, f)
with open('Forecasts/y_true_dict.pkl', 'wb') as f:
    pickle.dump(y_true_dict_bm, f)

In [53]:
# Placeholder for storing all values for each horizon and each model
predictions_dict_bm2 = {}
y_true_dict_bm2 = {}

# Loop over datasets
for model_name, (train, test) in Benchmark_datasets2.items():
    predictions_dict_bm2[model_name] = {h: [] for h in horizons}
    y_true_dict_bm2[model_name] = {h: [] for h in horizons}

    # Concatenate train and test
    data = np.concatenate([train, test])

    # Initialize the model outside the loop
    model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=tscv, max_iter=1000000, tol=0.0001)
    
    # Define a variable to keep track of the last observed value of the recession indicator
    last_indicator = None

    # Loop over horizons
    for h in horizons:
        y_true_per_bm2_horizon = []
        y_pred_per_bm2_horizon = []

        # Loop over time points in the test set
        for i in range(len(train) + h - 1, len(data)):
            # Get train and test data up to the forecast origin
            train_temp = data[:i - h + 1]
            test_temp = data[i - h + 1:i + 1]

            # Get the corresponding targets
            y_train_temp = Target[:i - h + 1]
            y_test_temp = Target[i - h + 1:i + 1]

            # Get the corresponding lag targets
            y_lag_train_temp = Lag_Target[:i - h + 1]
            y_lag_test_temp = Lag_Target[i - h + 1:i + 1]

            # Standardize the data
            scaler.fit(train_temp)
            train_temp_standardized = scaler.transform(train_temp)
            test_temp_standardized = scaler.transform(test_temp)
            
            # Check if we should update the model (i.e., if there is a change in the recession_indicator)
            current_indicator = updates[i]
            if  current_indicator != last_indicator:
                # Update the last observed value of the recession indicator
                last_indicator = current_indicator
                
                # Train the model
                model.fit(train_temp_standardized, np.ravel(y_train_temp))

                # If no features were selected, refit the model with a different l1_ratio
                refit_attempts = 0
                while len(np.nonzero(model.coef_)[0]) <= 1 and refit_attempts < 2:
                    model = ElasticNet(l1_ratio = 0.05, alpha = 0.05)
                    model.fit(train_temp_standardized, np.ravel(y_train_temp))
                    refit_attempts += 1

                #If still no features were selected after 2 attempts, print a warning
                if len(np.nonzero(model.coef_)[0]) <= 1:
                    print(f'Warning: Model failed to select more than one feature after {refit_attempts} attempts.')

            # If any features were selected, apply PCA
            if model.coef_.any():
                # Get indices of non-zero coefficients
                selected_features = np.nonzero(model.coef_)[0]
                selected_features_indices = np.nonzero(model.coef_)[0]
  
                # Select the features that were not discarded by the ElasticNet
                selected_train_temp = train_temp[:, selected_features]
                selected_test_temp = test_temp[:, selected_features]

                # Initialize and fit a new scaler on the selected features
                scaler_pca = StandardScaler()
                scaler_pca.fit(selected_train_temp)
                
                # Standardize selected features
                selected_train_temp_standardized = scaler_pca.transform(selected_train_temp)
                selected_test_temp_standardized = scaler_pca.transform(selected_test_temp)

                # Define PCA
                n_components = num_factors(selected_train_temp_standardized, kmax=8)  # Choose a suitable value for kmax
                pca = PCA(n_components= n_components)
                best_pca = pca.fit(selected_train_temp_standardized)

                # Transform data using the best PCA
                selected_train_temp_pca = best_pca.transform(selected_train_temp_standardized)
                selected_test_temp_pca = best_pca.transform(selected_test_temp_standardized)

                 # Add the lagged target as an additional column to the PCA-transformed data
                selected_train_temp_pca = np.column_stack((selected_train_temp_pca, y_lag_train_temp))
                selected_test_temp_pca = np.column_stack((selected_test_temp_pca, y_lag_test_temp))

                # Train a linear regression model and compute p-values
                lr = LinearRegression()

                # Calculate p-values
                mod = sm.OLS(np.ravel(y_train_temp), sm.add_constant(selected_train_temp_pca))
                fii = mod.fit()
                p_values = fii.summary2().tables[1]['P>|t|']

                # Find the significant features
                significant_features = p_values[p_values < 0.05].index  # Find features with p-value < 0.05

                # Ignore the constant term
                significant_features = [i for i in significant_features if i != 'const']

                # Create a mapping from column names to indices
                column_to_index = {col: idx-1 for idx, col in enumerate(fii.summary2().tables[1].index)}  # idx-1 corrects for the added constant

                # Convert column names to indices
                significant_indices = [column_to_index[col] for col in significant_features if column_to_index[col] != -1]  # We make sure not to include the constant

                # If there are significant features, retrain the model on these
                if significant_indices:
                    selected_train_temp_pca = selected_train_temp_pca[:, significant_indices]
                    selected_test_temp_pca = selected_test_temp_pca[:, significant_indices]
                else:
                    print("No features with p-value < 0.05 was found. Retaining all PCA-transformed features.")

                # Fit the model on the selected (or all) PCA-transformed features
                lr.fit(selected_train_temp_pca, np.ravel(y_train_temp))
                
                # Make a prediction and add it to the predictions list
                y_pred_pca_temp = lr.predict(selected_test_temp_pca)
                y_pred_per_bm2_horizon.append(y_pred_pca_temp[h-1]) # Remember python indexing starts from 0

                # Add true values to a list
                y_true_per_bm2_horizon.append(y_test_temp[h-1]) # Remember python indexing starts from 0

        predictions_dict_bm2[model_name][h] = y_pred_per_bm2_horizon
        y_true_dict_bm2[model_name][h] = y_true_per_bm2_horizon

# Save dictionaries to files for future use
with open('Forecasts/predictions_dict_bm2.pkl', 'wb') as f:
    pickle.dump(predictions_dict_bm2, f)